<a href="https://colab.research.google.com/github/FahmidaAkterShimu/Final-Project-using-NLP/blob/main/Hate_Speech_Detection_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hate Speech Detection using LSTM**

In [73]:
# Importing libraries
import pandas as pd
import spacy
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
df = pd.read_csv('/content/labeled_data.csv')

In [75]:
df.shape

(24783, 7)

In [76]:
df.head()

Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...

In [77]:
df.columns

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')

In [78]:
# 1. DELETING UNWANTED COLUMNS
df.drop(columns = ['Unnamed: 0',	'count',	'hate_speech',	'offensive_language',	'neither'],
        inplace = True)

In [79]:
df.head()

class                                              tweet
0      2  !!! RT @mayasolovely: As a woman you shouldn't...
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...

In [80]:
# 2. CHECKING FOR NULL VALUES
df.isna().sum()

class    0
tweet    0
dtype: int64

In [81]:
df['tweet'].iloc[0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [82]:
df['tweet'].iloc[100]

'"@ClicquotSuave: LMAOOOOOOOOOOO this nigga @Krillz_Nuh_Care http://t.co/AAnpSUjmYI" &lt;bitch want likes for some depressing shit..foh'

In [83]:
df['tweet'].iloc[1000]

'&#128514;&#128514;&#128514;&#128514;&#128514;&#128514;&#128514;"@betysweetcocker: That pussy is just....&#128561; imma assume she just had a baby like..the day before"'

In [84]:
# 3. REPLACING SYMBOLS AND NUMBERS FROM TEXT
df['processed_tweet'] = df['tweet'].str.replace(r'[^a-zA-Z]', ' ', regex = True)

In [85]:
df.head()

class                                              tweet  \
0      2  !!! RT @mayasolovely: As a woman you shouldn't...   
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

                                     processed_tweet  
0      RT  mayasolovely  As a woman you shouldn t...  
1        RT  mleew    boy dats cold   tyga dwn ba...  
2          RT  UrKindOfBrand Dawg     RT    sbaby...  
3            RT  C G Anderson   viva based she lo...  
4                RT  ShenikaRoberts  The shit you...

In [86]:
df['processed_tweet'].iloc[1000]

'                                                                 betysweetcocker  That pussy is just              imma assume she just had a baby like  the day before '

In [87]:
# 4. REPLACING MULTIPLE WHITESPACE CHARACTERS WITH SINGLE ONE
df['processed_tweet_2'] = df['processed_tweet'].str.replace(r'[\s]+', ' ', regex = True)

In [88]:
df.head()

class                                              tweet  \
0      2  !!! RT @mayasolovely: As a woman you shouldn't...   
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

                                     processed_tweet  \
0      RT  mayasolovely  As a woman you shouldn t...   
1        RT  mleew    boy dats cold   tyga dwn ba...   
2          RT  UrKindOfBrand Dawg     RT    sbaby...   
3            RT  C G Anderson   viva based she lo...   
4                RT  ShenikaRoberts  The shit you...   

                                   processed_tweet_2  
0   RT mayasolovely As a woman you shouldn t comp...  
1   RT mleew boy dats cold tyga dwn bad for cuffi...  
2   RT UrKindOfBrand Dawg RT sbaby life You ever ...  
3   RT C G Anderson viva based she look like a tr...  
4   RT ShenikaRoberts The shit you hear about me ...

In [89]:
df['processed_tweet_2'].iloc[1000]

' betysweetcocker That pussy is just imma assume she just had a baby like the day before '

In [90]:
# Deleting unwanted columns
df.drop(columns = ['tweet', 'processed_tweet'], inplace = True)

In [91]:
df.head()

class                                  processed_tweet_2
0      2   RT mayasolovely As a woman you shouldn t comp...
1      1   RT mleew boy dats cold tyga dwn bad for cuffi...
2      1   RT UrKindOfBrand Dawg RT sbaby life You ever ...
3      1   RT C G Anderson viva based she look like a tr...
4      1   RT ShenikaRoberts The shit you hear about me ...

In [92]:
# 5. REPLACING THE TARGET VALUES
# df['class'].replace({0 : 'hate', 1 : 'offensive', 2 : 'neither'}, inplace = True)

In [93]:
df.head()

class                                  processed_tweet_2
0      2   RT mayasolovely As a woman you shouldn t comp...
1      1   RT mleew boy dats cold tyga dwn bad for cuffi...
2      1   RT UrKindOfBrand Dawg RT sbaby life You ever ...
3      1   RT C G Anderson viva based she look like a tr...
4      1   RT ShenikaRoberts The shit you hear about me ...

In [94]:
# Load out spacy modeule
nlp = spacy.load('en_core_web_sm')

In [95]:
# 6. LEMMATIZATION
def lemmatization(text):
  doc = nlp(text)
  lemmaList = [word.lemma_ for word in doc]
  return ' '.join(lemmaList)

In [96]:
df['lemma_tweet'] = df['processed_tweet_2'].apply(lemmatization)

In [98]:
df.head()

class                                  processed_tweet_2  \
0      2   RT mayasolovely As a woman you shouldn t comp...   
1      1   RT mleew boy dats cold tyga dwn bad for cuffi...   
2      1   RT UrKindOfBrand Dawg RT sbaby life You ever ...   
3      1   RT C G Anderson viva based she look like a tr...   
4      1   RT ShenikaRoberts The shit you hear about me ...   

                                         lemma_tweet  
0    RT mayasolovely as a woman you shouldn t com...  
1    RT mleew boy dat cold tyga dwn bad for cuffi...  
2    RT UrKindOfBrand Dawg RT sbaby life you ever...  
3    RT C G Anderson viva base she look like a tr...  
4    RT ShenikaRoberts the shit you hear about I ...

In [99]:
df['processed_tweet_2'].iloc[67]

' Allyhaaaaa Lemmie eat a Oreo amp do these dishes One oreo Lol'

In [100]:
df['lemma_tweet'].iloc[67]

'  Allyhaaaaa Lemmie eat a Oreo amp do these dish one oreo lol'

In [101]:
# 7. REMOVING STOPWORDS
def remove_stopwords(text):
  doc = nlp(text)
  no_stopwords_list = [word.text for word in doc if not word.is_stop]
  return ' '.join(no_stopwords_list)

In [102]:
df['final_tweet'] = df['lemma_tweet'].apply(remove_stopwords)

In [103]:
df['lemma_tweet'].iloc[67]

'  Allyhaaaaa Lemmie eat a Oreo amp do these dish one oreo lol'

In [104]:
df['final_tweet'].iloc[67]

'   Allyhaaaaa Lemmie eat Oreo amp dish oreo lol'

In [105]:
df.head()

class                                  processed_tweet_2  \
0      2   RT mayasolovely As a woman you shouldn t comp...   
1      1   RT mleew boy dats cold tyga dwn bad for cuffi...   
2      1   RT UrKindOfBrand Dawg RT sbaby life You ever ...   
3      1   RT C G Anderson viva based she look like a tr...   
4      1   RT ShenikaRoberts The shit you hear about me ...   

                                         lemma_tweet  \
0    RT mayasolovely as a woman you shouldn t com...   
1    RT mleew boy dat cold tyga dwn bad for cuffi...   
2    RT UrKindOfBrand Dawg RT sbaby life you ever...   
3    RT C G Anderson viva base she look like a tr...   
4    RT ShenikaRoberts the shit you hear about I ...   

                                         final_tweet  
0     RT mayasolovely woman shouldn t complain cl...  
1     RT mleew boy dat cold tyga dwn bad cuffin d...  
2     RT UrKindOfBrand Dawg RT sbaby life fuck bi...  
3         RT C G Anderson viva base look like tranny  
4     RT ShenikaRoberts shit hear true faker bitc...

In [106]:
# 8. ONE - HOT REPRESENTATION

vocab_size = 10000
one_hot_representation = [one_hot(words, vocab_size) for words in df['final_tweet']]

In [107]:
df['final_tweet'].iloc[0]

'   RT mayasolovely woman shouldn t complain clean house amp man trash'

In [108]:
one_hot_representation[100]

[6434,
 1135,
 7252,
 8251,
 1238,
 8267,
 4319,
 2475,
 6260,
 9440,
 495,
 5342,
 806,
 7876,
 6133,
 4038,
 6892]

In [109]:
# 9. PADDING AND EMBEDDING

sentence_length = 20
embedded_tweet = pad_sequences(one_hot_representation, padding='pre', maxlen = sentence_length)

In [110]:
embedded_tweet[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 4997, 2385,
        399, 6736, 2475, 1210, 1388, 7323, 4549, 9276, 3272], dtype=int32)

In [112]:
for i in range (0,4):
  print(embedded_tweet[i])

[   0    0    0    0    0    0    0    0    0 4997 2385  399 6736 2475
 1210 1388 7323 4549 9276 3272]
[   0    0    0    0    0    0    0 4997 8173 6534 1110 1311 1906 4158
 2523  169 1110 7437 1263 5563]
[   0    0    0    0    0    0    0    0 4997 7956  148 4997 8468 2981
 8591 5342 9385 9783 6975 4038]
[   0    0    0    0    0    0    0    0    0    0    0 4997 9781 5340
 6474  522 6506 4038 7876 8024]


In [113]:
# 10. DIVIDING THE DATASET
X = np.array(embedded_tweet)
y = np.array(df['class'])

In [114]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [115]:
smote = SMOTE(sampling_strategy='minority')
X, y = smote.fit_resample(X, y)

In [116]:
# 11. TRAIN - TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [118]:
X.shape, X_train.shape, X_test.shape

((42543, 20), (34034, 20), (8509, 20))

In [119]:
# 12. CREATING MODEL

dimension = 50

model = keras.Sequential([
    #embedding layer
    keras.layers.Embedding(vocab_size, dimension, input_length = sentence_length),
    # LSTM layer
    keras.layers.LSTM(100,return_sequences=True),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50),
    #output layer
    keras.layers.Dense(3, activation = 'softmax')
])

#compilation
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [120]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [121]:
model.fit(X_train, y_train, epochs = 10, batch_size=32)

Epoch 1/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - accuracy: 0.7843 - loss: 0.4941
Epoch 2/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - accuracy: 0.9455 - loss: 0.1671
Epoch 3/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.9644 - loss: 0.1152
Epoch 4/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.9725 - loss: 0.0911
Epoch 5/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.9780 - loss: 0.0749
Epoch 6/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - accuracy: 0.9841 - loss: 0.0495
Epoch 7/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.9876 - loss: 0.0381
Epoch 8/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - accuracy: 0.9916 - loss: 0.0273
Epoch 9/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.9936 - loss: 0.0203
Epoch 10/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - accuracy: 0.9953 - loss: 0.0153


In [122]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy : {accuracy * 100}')

266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8949 - loss: 0.5737
Model Accuracy : 88.97637724876404


In [123]:
pred = np.argmax(model.predict(X_test), axis = -1)

266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step


In [124]:
y_test[:5]

array([1, 1, 0, 1, 2])

In [125]:
pred[:5]

array([1, 1, 0, 1, 2])

In [126]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3812
           1       0.90      0.91      0.91      3807
           2       0.77      0.66      0.71       890

    accuracy                           0.89      8509
   macro avg       0.86      0.83      0.84      8509
weighted avg       0.89      0.89      0.89      8509



In [127]:
cf = confusion_matrix(y_test, pred, normalize = 'true')
sns.heatmap(cf, annot = True, cmap = 'crest')
plt.xlabel('PREDICTED'),
plt.ylabel('ACTUAL');